In [1]:
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

def rate_url_validity(user_query: str, url: str) -> dict:
    """
    Evaluates the validity of a given URL by computing various metrics including
    domain trust, content relevance, fact-checking, bias, and citation scores.

    Args:
        user_query (str): The user's original query.
        url (str): The URL to analyze.

    Returns:
        dict: A dictionary containing scores for different validity aspects.
    """

    # === Step 1: Fetch Page Content ===
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        page_text = " ".join([p.text for p in soup.find_all("p")])  # Extract paragraph text
    except Exception as e:
        return {"error": f"Failed to fetch content: {str(e)}"}

    # === Step 2: Domain Authority Check (Moz API) ===
    # Replace with actual Moz API call
    domain_trust = 60  # Placeholder value (Scale: 0-100)

    # === Step 3: Content Relevance (Semantic Similarity using Hugging Face) ===
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    similarity_score = util.pytorch_cos_sim(model.encode(user_query), model.encode(page_text)).item() * 100

    # === Step 4: Fact-Checking (Google Fact Check API) ===
    fact_check_score = check_facts(page_text)

    # === Step 5: Bias Detection (NLP Sentiment Analysis) ===
    sentiment_pipeline = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment")
    sentiment_result = sentiment_pipeline(page_text[:512])[0]  # Process first 512 characters
    bias_score = 100 if sentiment_result["label"] == "POSITIVE" else 50 if sentiment_result["label"] == "NEUTRAL" else 30

    # === Step 6: Citation Check (Google Scholar via SerpAPI) ===
    citation_count = check_google_scholar(url)
    citation_score = min(citation_count * 10, 100)  # Normalize

    # === Step 7: Compute Final Validity Score ===
    final_score = (
        (0.3 * domain_trust) +
        (0.3 * similarity_score) +
        (0.2 * fact_check_score) +
        (0.1 * bias_score) +
        (0.1 * citation_score)
    )

    return {
        "Domain Trust": domain_trust,
        "Content Relevance": similarity_score,
        "Fact-Check Score": fact_check_score,
        "Bias Score": bias_score,
        "Citation Score": citation_score,
        "Final Validity Score": final_score
    }


# === Helper Function: Fact-Checking via Google API ===
def check_facts(text: str) -> int:
    """
    Cross-checks text against Google Fact Check API.
    Returns a score between 0-100 indicating factual reliability.
    """
    api_url = f"https://toolbox.google.com/factcheck/api/v1/claimsearch?query={text[:200]}"
    try:
        response = requests.get(api_url)
        data = response.json()
        if "claims" in data and data["claims"]:
            return 80  # If found in fact-checking database
        return 40  # No verification found
    except:
        return 50  # Default uncertainty score


# === Helper Function: Citation Count via Google Scholar API ===
def check_google_scholar(url: str) -> int:
    """
    Checks Google Scholar citations using SerpAPI.
    Returns the count of citations found.
    """
    serpapi_key = "API KEY"
    params = {"q": url, "engine": "google_scholar", "api_key": serpapi_key}
    try:
        response = requests.get("https://serpapi.com/search", params=params)
        data = response.json()
        return len(data.get("organic_results", []))
    except:
        return -1  # Assume no citations found

15 samples

In [2]:
user_prompt = "What is the weather report in jersey city in february 2025"
url_to_check = "https://www.weather25.com/north-america/usa/new-jersey/jersey-city?page=month&month=February#google_vignette"

result = rate_url_validity(user_prompt, url_to_check)
print(result)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 81.47368431091309, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 55.442105293273926}


In [3]:
user_prompt = "Tell something about the country India"
url_to_check = "https://en.wikipedia.org/wiki/India"

result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 74.01041984558105, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 53.203125953674316}


In [4]:
user_prompt = "What is linear regression"
url_to_check = "https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/what-is-linear-regression/"

result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 52.85009145736694, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 46.85502743721008}


In [5]:
user_prompt = "Importance of Data science"
url_to_check = "https://mate.academy/usa/courses/data-analytics-us?utm_source=google&utm_medium=cpc&utm_campaign=GEO-US_DataAnalytics_SEA_MatchType-All_P-TCPA&utm_content=KW-DS_course&utm_term=kwd-64263479108_b_data%20science%20bootcamp_g&utm_ac_id=179-791-8507&utm_id=22211763589&utm_ad_group_id=177055082800&utm_ad_id=732173289761&utm_term_id=kwd-64263479108&gad_source=1&gclid=Cj0KCQiA_NC9BhCkARIsABSnSTZ8KLmVnqyWLQMfcgrP81DaGY66kTWvpwjRgI_OKXw_yjS5hrMtwnIaAkDBEALw_wcB"

result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 32.260629534721375, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 40.67818886041641}


In [6]:
user_prompt = "best ways to become datascientist"
url_to_check = "https://www.coursera.org/articles/what-is-a-data-scientist"

result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 65.76454639434814, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 50.72936391830444}


In [7]:
user_prompt = "History of India"
url_to_check = "https://en.wikipedia.org/wiki/History_of_India"

result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 65.13030529022217, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 50.53909158706665}


In [8]:
user_prompt = "Most influencial novels of the year"
url_to_check = "https://lithub.com/the-20-best-novels-of-the-decade/"

result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 66.85273051261902, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 51.055819153785706}


In [9]:
user_prompt = "what is algorithms for datascience"
url_to_check = "https://www.geeksforgeeks.org/machine-learning-algorithms/"

result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 44.97482180595398, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 44.492446541786194}


In [10]:
user_prompt = "What role do algorithms play in the field of data science?"
url_to_check = "https://mentr-me.com/question/what-is-the-use-of-algorithms-in-data-science"

result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 71.55957818031311, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 52.46787345409393}


In [11]:
user_prompt = "Which is the best stock to invest now?"
url_to_check = "https://www.fool.com/investing/2025/02/15/3-no-brainer-growth-stocks-to-buy-with-100-right-n/"
result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 66.55580997467041, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 50.96674299240112}


In [12]:
user_prompt = "what is the importance of python in data scince"
url_to_check = "https://www.geeksforgeeks.org/python-for-data-science/"

result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 80.01725673675537, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 55.00517702102661}


In [13]:
user_prompt = "what are latest technologies in ai"
url_to_check = "https://today.ucsd.edu/story/7-ai-powered-technologies-you-should-know-about"
result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 46.30809426307678, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 44.892428278923035}


In [14]:
user_prompt = "how to invest in stocks"
url_to_check = "https://www.thewealthyinvestor.net/newyorkseminar/?gad_source=1"

result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 34.29941534996033, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 41.2898246049881}


In [15]:
user_prompt = "how to invest in crypto"
url_to_check = "https://www.schwab.com/cryptocurrency"
result = rate_url_validity(user_prompt, url_to_check)
print(result)

Device set to use cpu


{'Domain Trust': 60, 'Content Relevance': 5.655687302350998, 'Fact-Check Score': 50, 'Bias Score': 30, 'Citation Score': 0, 'Final Validity Score': 32.6967061907053}
